<a href="https://colab.research.google.com/github/viniciusmmartins/PosFiap-AI-TechChallange-Fase3/blob/main/Tech_Challange_fase3_RAG_Gemma_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG de protocolos médicos

## Dependency install

In [ ]:
!pip install transformers torch accelerate langchain langchain-community langchain-core langchain-chroma sentence-transformers chromadb huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.0 MB/s eta 0:00:0

## Imports

In [ ]:
import requests
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from huggingface_hub import upload_folder
import json
from google.colab import userdata

## Download Data from Your API

In [ ]:
url = "http://72.60.57.100:8000/protocols"
resp = requests.get(url)
resp.raise_for_status()
protocols = resp.json()

print(protocols)

[{'id': 'proto_0001', 'name': 'Stroke - Protocol 1', 'category': 'Toxicology', 'description': 'Standard protocol for handling Stroke in Toxicology context.', 'created_at': '2025-10-16T21:18:07.959960', 'steps': [{'step_number': 1, 'description': 'Step 1 description', 'action': 'Monitor ECG', 'duration_minutes': 8}, {'step_number': 2, 'description': 'Step 2 description', 'action': 'Check Vitals', 'duration_minutes': 41}, {'step_number': 3, 'description': 'Step 3 description', 'action': 'Check Blood Glucose', 'duration_minutes': 27}, {'step_number': 4, 'description': 'Step 4 description', 'action': 'Administer Antibiotics', 'duration_minutes': 7}, {'step_number': 5, 'description': 'Step 5 description', 'action': 'Order CT Scan', 'duration_minutes': 1}, {'step_number': 6, 'description': 'Step 6 description', 'action': 'Perform CPR', 'duration_minutes': 55}, {'step_number': 7, 'description': 'Step 7 description', 'action': 'Administer Oxygen', 'duration_minutes': 1}, {'step_number': 8, 'de

## Convert each protocol into a clean RAG text document

In [ ]:
def protocol_to_text(proto):
    """
    Converts your protocol JSON into a clean natural-language
    document for embedding in RAG.
    """
    header = (
        #f"Protocol ID: {proto['id']}\n"
        f"Name: {proto['name']}\n"
        f"Category: {proto['category']}\n"
        f"Description: {proto['description']}\n"
        f"Created at: {proto['created_at']}\n\n"
        "Steps:\n"
    )

    step_lines = []
    for step in proto.get("steps", []):
        step_lines.append(
            f"- Step {step['step_number']}: {step['description']}. "
            f"Action: {step['action']}. "
            f"Duration: {step['duration_minutes']} minutes."
        )

    return header + "\n".join(step_lines)


documents = [protocol_to_text(p) for p in protocols]

print(documents)

print("Total protocols loaded:", len(documents))

['Name: Stroke - Protocol 1\nCategory: Toxicology\nDescription: Standard protocol for handling Stroke in Toxicology context.\nCreated at: 2025-10-16T21:18:07.959960\n\nSteps:\n- Step 1: Step 1 description. Action: Monitor ECG. Duration: 8 minutes.\n- Step 2: Step 2 description. Action: Check Vitals. Duration: 41 minutes.\n- Step 3: Step 3 description. Action: Check Blood Glucose. Duration: 27 minutes.\n- Step 4: Step 4 description. Action: Administer Antibiotics. Duration: 7 minutes.\n- Step 5: Step 5 description. Action: Order CT Scan. Duration: 1 minutes.\n- Step 6: Step 6 description. Action: Perform CPR. Duration: 55 minutes.\n- Step 7: Step 7 description. Action: Administer Oxygen. Duration: 1 minutes.\n- Step 8: Step 8 description. Action: Check Blood Glucose. Duration: 17 minutes.', 'Name: Overdose - Protocol 2\nCategory: Neurology\nDescription: Standard protocol for handling Overdose in Neurology context.\nCreated at: 2025-02-01T21:18:08.057818\n\nSteps:\n- Step 1: Step 1 descr

## Chunk the Documents

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,      # Large enough for a full step + context
    chunk_overlap=100,   # High overlap to maintain continuity
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks = []
for doc in documents:
    chunks.extend(splitter.split_text(doc))

print(f"Generated {len(chunks)} chunks from {len(documents)} protocols.")

Generated 1980 chunks from 1000 protocols.


## Create Embeddings


In [ ]:
embed_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    model_kwargs={'device': 'cpu'}, # Change to 'cuda' if GPU available
    encode_kwargs={'normalize_embeddings': True}
)

/tmp/ipython-input-2636134415.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Create ChromaDB Vector Store

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
db_dir = "/content/drive/MyDrive/AI/tech challenge/rag/chroma_protocols_gemma7b"

client = chromadb.PersistentClient(path=db_dir)

collection = client.get_or_create_collection("protocols_collection")

batch_size = 100
total_chunks = len(chunks)

for i in range(0, total_chunks, batch_size):
    batch_chunks = chunks[i : i + batch_size]
    batch_ids = [f"chunk_{j}" for j in range(i, i + len(batch_chunks))]

    # Compute embeddings manually to ensure control
    batch_embeddings = embed_model.embed_documents(batch_chunks)

    collection.add(
        ids=batch_ids,
        documents=batch_chunks,
        embeddings=batch_embeddings
    )
    print(f"Processed batch {i} to {i+len(batch_chunks)}")

Processed batch 0 to 100
Processed batch 100 to 200
Processed batch 200 to 300
Processed batch 300 to 400
Processed batch 400 to 500
Processed batch 500 to 600
Processed batch 600 to 700
Processed batch 700 to 800
Processed batch 800 to 900
Processed batch 900 to 1000
Processed batch 1000 to 1100
Processed batch 1100 to 1200
Processed batch 1200 to 1300
Processed batch 1300 to 1400
Processed batch 1400 to 1500
Processed batch 1500 to 1600
Processed batch 1600 to 1700
Processed batch 1700 to 1800
Processed batch 1800 to 1900
Processed batch 1900 to 1980


## Upload to Hugging Face

In [ ]:
repo_id = "gerson-analista/rag-gemma-4b-tech-challenge"
folder_path = "./chroma"

upload_folder(
    folder_path=db_dir,
    repo_id=repo_id,
    repo_type="dataset",
    token=userdata.get('HF_TOKEN')
)

print("Uploaded to HuggingFace:", repo_id)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0-5c9674efcf89/header.bin: 100%|##########|   100B /   100B            

  ...ls_gemma7b/chroma.sqlite3:  45%|####5     | 3.79MB / 8.33MB            

  ...674efcf89/data_level0.bin:  45%|####5     | 1.46MB / 3.21MB            

  ...f89/index_metadata.pickle:  45%|####5     | 17.2kB / 37.9kB            

  ...0-5c9674efcf89/length.bin:  45%|####5     | 1.82kB / 4.00kB            

  ...9674efcf89/link_lists.bin:  45%|####5     | 3.92kB / 8.62kB            

Uploaded to HuggingFace: gerson-analista/rag-gemma-7b-tech-challenge


## Test: How to use RAG from hugging face

In [ ]:
from huggingface_hub import snapshot_download
from sentence_transformers import SentenceTransformer
import chromadb
import os

# --------------------------------------------------
# 1. Download dataset from Hugging Face
# --------------------------------------------------
folder = snapshot_download(
    repo_id="gerson-analista/rag-gemma-4b-tech-challenge",
    repo_type="dataset",
    token=userdata.get('HF_TOKEN'),  # optional if public
)

# --------------------------------------------------
# 2. Auto-detect Chroma DB path
# --------------------------------------------------
db_path = None
for root, dirs, files in os.walk(folder):
    if "chroma.sqlite3" in files:
        db_path = root
        break

if not db_path:
    raise RuntimeError("❌ Chroma database not found")

print(f"✅ Chroma DB found at: {db_path}")

# --------------------------------------------------
# 3. Initialize Chroma Persistent Client
# --------------------------------------------------
client = chromadb.PersistentClient(path=db_path)

collections = client.list_collections()
print("📦 Available collections:", collections)

collection = client.get_collection(name="protocols_collection")

print(f"📄 Total documents in collection: {collection.count()}")

# --------------------------------------------------
# 4. Load EMBEDDING MODEL (MUST MATCH INDEX)
# --------------------------------------------------
# ✅ Dataset was indexed using BGE
embed_model = SentenceTransformer(
    "BAAI/bge-base-en-v1.5",
    device="cpu"
)

# 5. Query
query = "List steps to stroke protocols"
q_emb = embed_model.encode([query]).tolist()

# 6. Perform retrieval
results = collection.query(
    query_embeddings=q_emb,
    n_results=3
)

print(results)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

4d98f51b-ff2a-453b-bacc-3ec03b2201aa/hea(…):   0%|          | 0.00/100 [00:00<?, ?B/s]

4d98f51b-ff2a-453b-bacc-3ec03b2201aa/len(…):   0%|          | 0.00/4.00k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

4d98f51b-ff2a-453b-bacc-3ec03b2201aa/ind(…):   0%|          | 0.00/37.9k [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'ids': [['chunk_866', 'chunk_1193', 'chunk_1009']], 'embeddings': None, 'documents': [['Name: Stroke - Protocol 431\nCategory: Cardiology\nDescription: Standard protocol for handling Stroke in Cardiology context.\nCreated at: 2025-09-01T21:18:08.064787\n\nSteps:\n- Step 1: Step 1 description. Action: Monitor ECG. Duration: 37 minutes.\n- Step 2: Step 2 description. Action: Check Blood Glucose. Duration: 47 minutes.\n- Step 3: Step 3 description. Action: Start IV Fluids. Duration: 20 minutes.\n- Step 4: Step 4 description. Action: Perform CPR. Duration: 15 minutes.', 'Name: Stroke - Protocol 600\nCategory: Cardiology\nDescription: Standard protocol for handling Stroke in Cardiology context.\nCreated at: 2025-08-22T21:18:08.068616\n\nSteps:\n- Step 1: Step 1 description. Action: Check Blood Glucose. Duration: 58 minutes.\n- Step 2: Step 2 description. Action: Give Aspirin. Duration: 2 minutes.\n- Step 3: Step 3 description. Action: Administer Oxygen. Duration: 12 minutes.', 'Name: Strok